In [ ]:
# System & Python dependencies
!apt -y update -qq
!apt -y install -qq libgl1-mesa-glx wget git

# Clone ComfyUI
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI

# Install Python packages
!pip install -r requirements.txt

import torch
print("✅ CUDA available:", torch.cuda.is_available())
print("💡 GPU device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


In [ ]:
import os
from google.colab import drive

# 1. Conectar Google Drive
drive.mount('/content/drive')

# 2. Definir rutas Clave
base_path = "/content"
comfy_path = f"{base_path}/ComfyUI"

# --- RUTAS EN TU GOOGLE DRIVE (Lo que se queda guardado) ---
drive_output_path = "/content/drive/MyDrive/ComfyUI_Output"       # Imágenes
drive_workflows_path = "/content/drive/MyDrive/ComfyUI_Workflows" # Flujos .json

# 3. Instalar Herramientas de Descarga
!apt-get -y install -qq aria2

# 4. Instalar ComfyUI en disco local (Efímero)
if not os.path.exists(comfy_path):
    print("⚡ Instalando ComfyUI en el disco local de Colab...")
    %cd {base_path}
    !git clone https://github.com/comfyanonymous/ComfyUI
    %cd {comfy_path}
    !pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu121

    # Instalar Manager y soporte GGUF
    %cd custom_nodes
    !git clone https://github.com/ltdrdata/ComfyUI-Manager
    !git clone https://github.com/city96/ComfyUI-GGUF
    !pip install -r ComfyUI-GGUF/requirements.txt
    %cd ..
else:
    print("✅ ComfyUI ya está instalado.")

# ==============================================================================
# 🔗 VINCULACIÓN A DRIVE (La parte importante)
# ==============================================================================

# A) VINCULAR IMÁGENES (Output -> Drive)
if not os.path.exists(drive_output_path):
    os.makedirs(drive_output_path, exist_ok=True)

if os.path.exists(f"{comfy_path}/output"):
    !rm -rf {comfy_path}/output  # Borramos la carpeta local vacía
!ln -s {drive_output_path} {comfy_path}/output # Creamos el enlace a Drive

# B) VINCULAR WORKFLOWS (User/Workflows -> Drive)
# Esta es la ruta interna donde ComfyUI busca los "Saved Workflows"
local_workflows_path = f"{comfy_path}/user/default/workflows"

# 1. Asegurar que la carpeta existe en Drive
if not os.path.exists(drive_workflows_path):
    os.makedirs(drive_workflows_path, exist_ok=True)
    print("📂 Carpeta 'ComfyUI_Workflows' creada en tu Drive.")

# 2. Asegurar que la ruta padre local existe
os.makedirs(f"{comfy_path}/user/default", exist_ok=True)

# 3. Si existe una carpeta local de workflows (por defecto), la eliminamos
if os.path.exists(local_workflows_path):
    !rm -rf {local_workflows_path}

# 4. Creamos el enlace simbólico: ComfyUI creerá que guarda en local, pero va a Drive
!ln -s {drive_workflows_path} {local_workflows_path}

print("\n" + "★"*50)
print("✅ SISTEMA CONECTADO A GOOGLE DRIVE:")
print(f"   📂 Tus Imágenes irán a: {drive_output_path}")
print(f"   📂 Tus Workflows irán a: {drive_workflows_path}")
print("★"*50 + "\n")

In [ ]:
COMFYUI_PATH = '/content/ComfyUI'

LINKS = {
    'models': f'{GDRIVE_BASE}/models',
    'custom_nodes': f'{GDRIVE_BASE}/custom_nodes',
    'input': f'{GDRIVE_BASE}/input',
    'output': f'{GDRIVE_BASE}/output',
    'temp': f'{GDRIVE_BASE}/temp',
    'user': f'{GDRIVE_BASE}/user'
}

for name, target in LINKS.items():
    source = os.path.join(COMFYUI_PATH, name)
    # Remove old local folder
    if os.path.islink(source) or os.path.isdir(source):
        !rm -rf "{source}"
    # Create symlink
    os.symlink(target, source)

print("✅ ComfyUI is now fully configured to use Google Drive.")

In [ ]:
import os

# ==============================================================================
# 📋 LISTA MAESTRA DE MODELOS (AGREGA TUS LINKS AQUÍ)
# ==============================================================================
# Formato: ("URL_DE_DESCARGA", "NOMBRE_DEL_ARCHIVO.extensión", "TIPO_DE_CARPETA")
# Tipos válidos: "unet", "clip", "vae", "lora", "checkpoint", "controlnet"

DOWNLOAD_LIST = [
    # --- MODELOS FLUX BÁSICOS (No borrar) ---
    ("https://huggingface.co/city96/FLUX.1-dev-gguf/resolve/main/flux1-dev-Q4_K_S.gguf", "flux1-dev-Q4_K_S.gguf", "unet"),
    ("https://huggingface.co/city96/t5-v1_1-xxl-encoder-gguf/resolve/main/t5-v1_1-xxl-encoder-Q4_K_S.gguf", "t5-v1_1-xxl-encoder-Q4_K_S.gguf", "clip"),
    ("https://huggingface.co/f5aiteam/CLIP/resolve/main/clip_l.safetensors", "clip_l.safetensors", "clip"),
    ("https://huggingface.co/f5aiteam/VAE/resolve/main/ae.safetensors", "ae.safetensors", "vae"),

    # --- T5 y FLUX FILL (Los que te faltaban antes) ---
    ("https://huggingface.co/comfyanonymous/flux_text_encoders/resolve/main/t5xxl_fp16.safetensors", "t5xxl_fp16.safetensors", "clip"),
    ("https://huggingface.co/Kijai/flux-fp8/resolve/main/flux1-fill-dev-fp8.safetensors", "flux1-fill-dev-fp8.safetensors", "unet"),

    # --- TUS LORAS Y CHECKPOINTS DE CIVITAI ---
    # Nota: Recuerda poner tu token si el link lo requiere
    ("https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=a0a09fdb343a3ec29b67c011e464a1d7", "NSFW_master.safetensors", "lora"),
    ("https://civitai.com/api/download/models/1198851?type=Model&format=SafeTensor&token=a0a09fdb343a3ec29b67c011e464a1d7", "flux1_devFP8Kijai11GB.safetensors", "checkpoint"),

    # --- AGREGA NUEVOS AQUÍ ABAJO (Copia y pega una línea anterior) ---
    # ("URL", "NOMBRE.safetensors", "lora"),

]

# ==============================================================================
# ⚙️ MOTOR DE DESCARGA (NO TOCAR ESTA PARTE)
# ==============================================================================
print("🚀 Iniciando Gestor de Descargas Inteligente...")

# Definir rutas base
base_path = "/content/ComfyUI/models"
folders = {
    "unet": f"{base_path}/unet",
    "clip": f"{base_path}/clip",
    "vae": f"{base_path}/vae",
    "lora": f"{base_path}/loras",
    "checkpoint": f"{base_path}/checkpoints",
    "controlnet": f"{base_path}/controlnet",
    "text_encoder": f"{base_path}/clip" # Alias para compatibilidad
}

# Crear carpetas necesarias
for path in folders.values():
    os.makedirs(path, exist_ok=True)

# Bucle de descarga
for url, filename, type_key in DOWNLOAD_LIST:
    # Verificar si el tipo es válido
    if type_key not in folders:
        print(f"⚠️ Tipo desconocido: '{type_key}' para {filename}. Saltando...")
        continue

    destination = folders[type_key]
    file_path = os.path.join(destination, filename)

    # Comprobar si ya existe para no descargar doble (útil si re-ejecutas la celda)
    if os.path.exists(file_path):
        print(f"✅ Ya existe: {filename}")
    else:
        print(f"⬇️ Descargando: {filename} en [{type_key}]...")
        # Ejecutar Aria2
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -d "{destination}" -o "{filename}"

print("\n✨ ¡Todo listo! Modelos actualizados.")

In [ ]:
import os

# ✅ CONFIG
drive_model_path = "/content/drive/MyDrive/ComfyUI/models/checkpoints"
model_filename = "sd_xl_base_1.0.safetensors"
model_url = "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"
force_refresh = False  # 🔁 Set to True to redownload even if file exists

# ✅ Construct full path
model_path = os.path.join(drive_model_path, model_filename)

# ✅ Ensure model folder exists
os.makedirs(drive_model_path, exist_ok=True)

# ✅ Logic to download or refresh
if not os.path.exists(model_path):
    print("⬇️  Model not found in Google Drive. Downloading...")
    !wget -O "{model_path}" "{model_url}"
    print("✅ Model downloaded to Google Drive.")
elif force_refresh:
    print("🔁 Force refresh is enabled. Re-downloading model...")
    !wget -O "{model_path}" "{model_url}"
    print("✅ Model refreshed in Google Drive.")
else:
    print("✅ Model already exists in Google Drive. Skipping download.")


In [ ]:
!rm -f ngrok ngrok.zip ngrok-stable-linux-amd64.tgz
!wget -O ngrok-stable-linux-amd64.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-stable-linux-amd64.tgz
!chmod +x ngrok
!./ngrok version
!./ngrok authtoken PUT YOUR TOKEN HERE


In [ ]:
# ComfyUI Manager
!pip install "numpy==1.26.4" --force-reinstall --quiet > /dev/null 2>&1
!rm -rf /content/ComfyUI/custom_nodes/ComfyUI-Manager
!git clone --quiet https://github.com/ltdrdata/ComfyUI-Manager.git /content/ComfyUI/custom_nodes/ComfyUI-Manager
print("✅ ComfyUI-Manager installed and numpy version is set for compatibility.")


In [ ]:
import subprocess
import time
import requests

# Start ngrok tunnel to port 8188
ngrok_process = subprocess.Popen(['./ngrok', 'http', '8188'],
                                 stdout=subprocess.DEVNULL,
                                 stderr=subprocess.STDOUT)

# Wait and retry connection
for i in range(10):
    try:
        r = requests.get('http://localhost:4040/api/tunnels')
        public_url = r.json()['tunnels'][0]['public_url']
        print(f"✅ Ngrok tunnel established: {public_url}")
        break
    except Exception as e:
        print(f"⏳ Attempt {i+1}/10: Ngrok not ready yet...")
        time.sleep(2)
else:
    print("❌ Ngrok failed to start. Please restart the runtime and try again.")



In [ ]:
# Start ComfyUI with public access
%cd /content/ComfyUI
!python main.py --listen 0.0.0.0 --port 8188   --cuda-device 0
